In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget

import numpy
from matplotlib import pyplot

# Loading and visualizing datasets

## Collection of included datasets

In [3]:
from pathlib import Path

metadata_paths = {
    'sim_mos2': Path('sample_data/simulated_mos2/mos2_0.00_dstep1.0.json'),
    'exp_mos2': Path('sample_data/experimental_mos2/mos2_twisted.json'),
    'exp_si': Path('sample_data/experimental_si/acq12_20over.json'),
    'sim_si': Path('sample_data/simulated_si/Si_110_Sn_300kV_conv25_defocus15_tds_199.70_dstep0.6.json')
}

for path in metadata_paths.values():
    assert path.exists()

## View dataset metadata

In [4]:
# load metadata for a dataset
import json

meta_path = metadata_paths['exp_si']

with open(meta_path, 'r') as f:
    metadata = json.load(f)

metadata

## Load raw data

In [5]:
from phaser.io.empad import load_4d

raw_data = load_4d(meta_path.parent / metadata['raw_filename'])
raw_data.shape

## Visualize raw data

Here, we visualize the sum of the diffraction patterns. This is a position-averaged convergent beam electron diffraction (PACBED) pattern, and can be used to accurately identify crystalline thickness and mistilt

In [7]:
from phaser.utils.plotting import plot_pacbed

plot_pacbed(raw_data, log=True, diff_step=metadata['diff_step']);

We can also visualize the dataset using virtual images and the raw diffraction patterns. This is useful for identifying data orientation, detector rotation, defocus, and overall dataset quality

In [8]:
from phaser.utils.plotting import plot_raw

ky = numpy.arange(raw_data.shape[-2], dtype=numpy.float32) - raw_data.shape[-2] / 2.
kx = numpy.arange(raw_data.shape[-1], dtype=numpy.float32) - raw_data.shape[-1] / 2.
ky, kx = numpy.meshgrid(numpy.fft.ifftshift(ky), numpy.fft.ifftshift(kx), indexing='ij')
k2 = ky**2 + kx**2

# create a virtual detector
# in this case this is a 3 px virtual bright field (vBF) detector
mask = k2 <= 3**2

plot_raw(
    raw_data, mask=mask,
    scan_step=tuple(s*1e10 for s in metadata['scan_step']),
    diff_step=metadata['diff_step'], log=False,
);

# Analyzing sampling metrics

To further assess the quality of the dataset, we can use sampling metrics. These metrics are a useful rule of thumb for identifying optimal acquisition parameters.

In [9]:
from phaser.utils.optics import calc_metrics
from phaser.utils.plotting import plot_metrics

metrics = calc_metrics(
    voltage=metadata['voltage'], conv_angle=metadata['conv_angle'],
    defocus=metadata['defocus']*1e10, scan_step=metadata['scan_step'][0]*1e10,
    diff_step=metadata['diff_step'], threshold=0.3
)
metrics

In [10]:
plot_metrics(metrics);

# Running a reconstruction

First, we start a reconstruction manager, which contains reconstruction workers:

In [1]:
from phaser.web.notebook import Manager
from phaser.plan import ReconsPlan

manager = Manager()
manager.start()

Then, we can submit a job:

Reconstructions are specified in a declarative format, and then processed by the worker to produce a reconstruction

In [17]:
mos2_meta

In [6]:
mos2_meta_path = metadata_paths['sim_mos2']

with open(mos2_meta_path, 'r') as f:
    mos2_meta = json.load(f)

plan = ReconsPlan.from_data({
    'name': 'mos2_lsqml',

    # how to load the raw data
    'raw_data': {
        'type': 'empad',
        'path': mos2_meta_path.parent / mos2_meta['raw_filename'],
        'diff_step': mos2_meta['diff_step'],
        'kv': mos2_meta['voltage'] / 1e3,
    },

    # how to initialize the simulation
    'init_probe': {
        'type': 'focused',
        'conv_angle': mos2_meta['conv_angle'],
        'defocus': mos2_meta['defocus'] * 1e10,
    },
    'init_object': 'random',
    'init_scan': {
        'type': 'raster',
        'shape': mos2_meta['scan_shape'],
        'step_size': mos2_meta['scan_step'][0] * 1e10,
    },
    'preprocessing': [{
        'type': 'poisson',
        'scale': 1.0e+6,
    }],

    # how to run the reconstruction
    'engines': [{
        'type': 'conventional',
        'probe_modes': 4,
        'niter': 200,
        'grouping': 64,
        'noise_model': {
            'type': 'anscombe',
            'eps': 0.1,
        },
        'solver': {
            'type': 'lsqml',
            'gamma': 1.0e-4,
            'illum_reg_object': 1.0e-2,
            'illum_reg_probe': 1.0e-2,
        },
        'regularizers': [{
            'type': 'clamp_object_amplitude',
            'amplitude': 1.2,
        }],
        'update_probe': {'after': 1},
        'update_positions': False,
    }]
})

plan

In [ ]:
manager.start_job(plan)